# Basic CSP Startup Script

Before starting the notebook, please run the job `deploy-sut-integration` of the `integration` stage in the latest gitlab pipeline: https://gitlab.com/ska-telescope/ska-mid-itf/-/pipelines?page=1&scope=all&ref=main. 

<div class="alert alert-block alert-danger">
<b>WARNING</b>The CBF configuration scripts' output is extremely verbose and will cause the CI job outputs to be too large to download in a single CI job. In future this might be improved.</div>

In [1]:
import os
import datetime, json
from tango import DeviceProxy, Database

# Take the namespace name from the deployment job
# KUBE_NAMESPACE = "ci-ska-mid-itf-at-1702-add-csplmc-tests"
KUBE_NAMESPACE = "integration"
KUBE_NAMESPACE_SDP = "integration-sdp"
CLUSTER_DOMAIN = "miditf.internal.skao.int"
# set the name of the databaseds service
DATABASEDS_NAME = "tango-databaseds"

tango_host = f"{DATABASEDS_NAME}.{KUBE_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"
print("Tango host %s" % tango_host)

# finally set the TANGO_HOST
os.environ["TANGO_HOST"] = tango_host

TIMEOUT=60

Tango host tango-databaseds.integration.svc.miditf.internal.skao.int:10000


List all devices available:

In [11]:
database = Database()
instance_list = database.get_device_exported("*")
print("%d devices avalable" % (len(instance_list)))
for instance in instance_list.value_string:
    #dev = DeviceProxy(instance)
    print(instance)
    #print("*****")
    #print(dev.info())

245 devices avalable
dserver/CbfController/controller
dserver/CbfSubarray/cbfsubarray-01
dserver/CbfSubarray/cbfsubarray-02
dserver/CbfSubarray/cbfsubarray-03
dserver/central_node_mid/01
dserver/csp_master_leaf_node/01
dserver/csp_subarray_leaf_node_mid/01
dserver/DataBaseds/2
dserver/dish_leaf_node/01
dserver/dish_leaf_node/100
dserver/dish_leaf_node/36
dserver/dish_leaf_node/63
dserver/ds-histogram/talon1_test
dserver/ds-resampler-delay-tracker/talon1_test
dserver/dscircuitswitch/talon1_test
dserver/dscorrelator/talon1_test
dserver/dsdct/talon1_test
dserver/dsfinechannelizer/talon1_test
dserver/dsgaussiannoisegen/talon1_test
dserver/dshostlutstage1/talon1_test
dserver/dshostlutstage2/talon1_test
dserver/dshpsmaster/talon1_test
dserver/dslstvbandpassfilter/talon1_test
dserver/dslstvgen/talon1_test
dserver/dslstvplayback/talon1_test
dserver/dslstvtonegenerator/talon1_test
dserver/dspacketstreamrepair/talon1_test
dserver/dspolarizationcoupler/talon1_test
dserver/dsslimtxrx/talon1_test
d

## Setup Device connection and Timeouts
Set up DeviceProxy, check AdminMode and if not 0, set to 0 (ONLINE)

In [12]:
CSP = DeviceProxy("mid-csp/control/0")

In [13]:
CSP.adminmode # This is likely currently OFFLINE

<adminMode.ONLINE: 0>

In [14]:
CSP.State() # This should now be DISABLE

tango._tango.DevState.ON

Set Adminmode to ONLINE and check state

In [15]:
CSP.adminmode=0
CSP.adminmode

<adminMode.ONLINE: 0>

In [16]:
CSP.State() # This should now be OFF

tango._tango.DevState.ON

Check CBF SimulationMode (this should be FALSE for real hardware control)

In [17]:
CSP.cbfSimulationMode # Should be True if hardware is not currently under CSP.LMC control

False

In [18]:
# CSP.cbfSimulationMode=False
CSP.cbfSimulationMode=True
CSP.cbfSimulationMode

True

Set Timeout to 60 seconds as the ON command is a long-running command and check value

In [19]:
CSP.commandTimeout=TIMEOUT
CSP.commandTimeout

60

## Check all relevant device states
Now check if all the devices are switched on as they should be.
Use the [Taranta Dashboard](https://k8s.miditf.internal.skao.int/integration/taranta/dashboard?id=6543b36cf6d53e0019b95cf1&mode=run) to monitor top-level devices!

## Send command sequence
### Send the ON command. 
NOTE: an empty list sends the ON command to ALL the subsystems, specific subsystems are turned on if specified in a list of subsystem FQDNs

In [ ]:
CSP.on([])

In [ ]:
CSP.State() # this should now be ON

### Control the CSP Subarray
Set up a Tango DeviceProxy to the CSP Subarray device


In [20]:
CSPSubarray=DeviceProxy('dserver/TmCspSubarrayLeafNodeTest/tm')

### Send AssignResource

In [ ]:
resources=json.dumps({
    "subarray_id": 1,
    "dish":{
        "receptor_ids":["SKA001"]
    }
})

CSPSubarray.AssignResources(resources)

### Send ConfigureScan command
NOTE: get the IP address of the interface of the SDP Surrogate pod and modify the JSON for the configure scan with it, before you run this command!!!

#### Copy this JSON and modify 
Uncomment the first line after getting it from the kubectl command output and modifying SDP_HOST_IP_ADDRESS
e.g. the IP address of the `pod cloud03-sdn-dynamic-vis` in the namespace `integration`

In [ ]:
SDP_HOST_IP_ADDRESS="10.10.41.60" # this is the IP address of the receiver interface
configuredata={ "interface": "https://schema.skao.int/ska-csp-configure/2.3", "subarray": { "subarray_name": "Single receptor" }, "common": { "config_id": "1 receptor, band 1, 1 FSP, no options", "frequency_band": "1", "subarray_id": 0 }, "cbf": { "delay_model_subscription_point": "ska_mid/tm_leaf_node/csp_subarray_01/delayModel", "rfi_flagging_mask": {}, "fsp": [ { "fsp_id": 1, "function_mode": "CORR", "receptors": ["SKA001"], "frequency_slice_id": 1, "zoom_factor": 1, "zoom_window_tuning": 450000, "integration_factor": 10, "channel_offset": 14880, "output_link_map": [ [0, 4], [744, 8], [1488, 12], [2232, 16], [2976, 20], [3720, 24], [4464, 28], [5206, 32], [5952, 36], [6696, 40], [7440, 44], [8184, 48], [8928, 52], [9672, 56], [10416, 60], [11160, 64], [11904, 68], [12648, 72], [13392, 76], [14136, 80] ], "output_host": [[0, "10.165.20.31"]], "output_port": [[0, 14000, 1]] } ] } }
configuredata["cbf"]["fsp"][0]["output_host"] = SDP_HOST_IP_ADDRESS

In [ ]:
CSPSubarray.ConfigureScan(json.dumps(configuredata))

## Setup BITE data stream
Copy the output of the next print line and paste into your shell.

In [ ]:
print(f"kubectl -n {KUBE_NAMESPACE} exec ec-bite -- python3 midcbf_bite.py --talon-bite-lstv-replay --boards=1")

### Upload the Delaymodel


In [ ]:
# Generate the Delaymodel and check if it was was correctly sent:
sub = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray_01")
current_time = float(datetime.datetime.utcnow().replace(tzinfo=datetime.timezone.utc).timestamp())
dm={"interface": "https://schema.skao.int/ska-csp-delaymodel/2.2", "epoch": current_time, "validity_period": 400.0, "delay_details": [{"receptor": "SKA001", "poly_info": [{"polarization": "X", "coeffs": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, {"polarization": "Y", "coeffs": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}]}]}
sub.delayModel=json.dumps(dm)
assert sub.delayModel==json.dumps(dm), f"Expected {dm}, got\n{sub.delayModel}"

## Scan
By now, we are supposed to look at the output of `tcpdump -i net1` in the SDP Surrogate pod and see data!

## End Scan (CSP Subarray)
If this fails because things were in READY, that means you didn't need to run it now - DON'T PANIC!

Run this if device is in 

In [ ]:
CSPSubarray.EndScan()

## Go To Idle (CSP Subarray)
CSPSubarray.GoToIdle()

In [ ]:
CSPSubarray.obsState

## Release Resources (CSP Subarray)
### Run either ReleaseAllResources, or ReleaseResources(<samejsonfilefortheothercommand>)

In [ ]:
CSPSubarray.ReleaseAllResources()

## Final Teardown
NOTE: This will off the CSP and CBF and should only be done if you don't want to use the system again.

In [ ]:
CSP.off([]) # Check with make itf-cbf-talonlru-status - lru should be off now
CSP.cbfSimulationMode=True
CSP.commandTimeout=3
CSP.adminmode=1